# Nagel–Schreckenberg model

Nagel–Schreckenberg model is a simple model used for modelling highway traffic.
#### Road
A road consists of cells. Each cell represents a single car or empty space on the road. 
#### Car 
Each car has a velocity (originally 0-5 units).
#### Transisition function
The transition function performs the following steps (order matters):
* Acceleration: The velocity of all cars having a velocity lower than the maximum velocity is increased by 1.
* Slowing down: If the distance between the current and the following car is smaller than the car's velocity, the velocity is reduced to the number of empty cells in front of the car (avoiding collisions).
* Randomization: The speed of all cars that have a *velocity >= 1* is reduced by one unit with the probability of *p*. 
* Car motion: All cars are moved forward by the *velocity* cells.

(source and more detailed description: [here](https://en.wikipedia.org/wiki/Nagel–Schreckenberg_model))

## Python implementation of Nagel-Schreckenberg model (6 points)

* Create a working Nagel-Schreckenberg model using the following template (4 points).
* Add periodic boundaries (1 point).
* Add a function that will randomly add new cars at the beginning of the road (1 point).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
import random
from IPython.display import HTML

In [2]:
%%capture
# Parameters
roadLength = 100
minVelocity = 0
maxVelocity = 5

# Number of steps 
steps = 100

rand_probability = 0.5 

# Config
fig, ax = plt.subplots(figsize=(24,4))
plt.axis('off')

In [3]:
class Car:
    distance = 0
    
    def __init__(self, minVelocity, maxVelocity):
        import random
        self.color = random.randint(0,200)
        self.velocity = random.randint(minVelocity,maxVelocity)
    
    #TODO Implement increaseVelcoity and randomizeVelocity functions 
    def increaseVelocity(self,maxVelocity):
        if ((self.velocity+1) <= maxVelocity):
            self.velocity += 1
        return
    
    def decreaseVelocity(self,minVelocity):
        if ((self.velocity-1) >= minVelocity):
            self.velocity -= 1
        return
    
    def setVelocity(self, newVelocity,maxVelocity):
        if newVelocity <= maxVelocity:
            self.velocity = newVelocity
        
    def randomizeVelocity(self, probability):
        if self.velocity >= 1:
            if random.random() < probability:
                self.velocity -= 1
        return
                
    def getColor(self):
        return self.color
    
    def isEmpty(self):
        return false
    
    def setDistance(self,distance):
        self.distance = distance

class Road:
    def __init__(self):
        return
    def getColor(self):
        return 255
    def isEmpty(self):
        return true

In [4]:
def initGrid(roadLength):
    # TODO Randomly initialize grid here (remove existing code)
    # grid should be a list
    grid = [Road()] * roadLength
    init_cars = random.sample(range(roadLength), 20)
    for car in init_cars:
        grid[car] = Car(minVelocity, maxVelocity)
            
    return grid

def show(grid, im):
        im.set_data(np.array(list(map(lambda x : x.getColor(), grid))).reshape(1,roadLength))

def getDistanceAhead(i, grid):
    dist = 0
    j = (i + 1) % roadLength
    while True:
        if type(grid[j]) is Car:
            return dist
        else:
            dist += 1
            j = (j + 1) % roadLength
    return dist             

def getNewPosition(position, vel):
    newPosition = position + vel
    if newPosition < roadLength:
        return newPosition
    else:
        return newPosition - roadLength
    if newPosition < 0:
        return roadLength - newPosition

def update(frameNum, grid, im, roadLength):
    
    if(frameNum == 0):
        show(grid,im)
        
    else:
        # 1.Acceleration
        for i in grid:
            if type(i) is Car:
                i.increaseVelocity(maxVelocity)

        # 2. Implement Slow down
        for i in range(len(grid)):
            if type(grid[i]) is Car:
                dist = getDistanceAhead(i, grid)
                if dist < grid[i].velocity:
                    grid[i].setVelocity(dist, maxVelocity)

        # 3. Randomization
        for i in grid:
            if type(i) is Car:
                i.randomizeVelocity(rand_probability)

        # 4. Implement Car motion
        newGrid = [Road()] * roadLength
        for idx, i in enumerate(grid):
            if type(i) is Car:
                newPosition = (idx + i.velocity) % roadLength
                newGrid[newPosition] = i
                
        if type(newGrid[0]) is not Car:
            if random.random() < 0.2:
                newGrid[0] = Car(minVelocity, maxVelocity)
        
        grid[:] = newGrid[:] 

        show(grid, im)
    return im,



In [5]:
grid = initGrid(roadLength)
im = ax.imshow(np.array(list(map(lambda x : x.getColor(), grid))).reshape(1,roadLength),cmap = 'gist_ncar')
ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [grid, im, roadLength])

HTML(ani.to_jshtml())


